In [ ]:
#!pip install hanlp
#!pip install simalign

In [124]:
import os, csv

import pandas as pd
from tqdm import tqdm
from simalign import SentenceAligner
# from google.cloud import translate_v2 as translate

In [272]:

src_lan = 'en'
# target language
tgt_lan = 'ES'
language = "Spanish"

RESOURCE = '../..'
DATA =  RESOURCE+'/data/'+language+'/aligned_2017'
TRANS_FILE = os.path.join(DATA, 'gold-sentences-'+language+'.tsv')
TOKENS_FILE = os.path.join(DATA, 'tokens-gold-'+language+'.tsv')
ALIGN_SENT_FILE = os.path.join(DATA, 'sentences-aligned-'+language+'.tsv')
ALIGN_TOKEN_FILE = os.path.join(DATA, 'tokens-aligned-'+language+'.tsv')

# DATA = os.path.join(RESOURCE, 'data')
DATA =  RESOURCE+'/data/English/2017'

KEY = os.path.join(RESOURCE, 'key')
RAW_SENSE_TSV = os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
RAW_TOKENS_TSV =  os.path.join(DATA, 'tokens-gold-English.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'gold-sentences-English.tsv')
JSON_GOOGLE_APPLICATION_CREDENTIALS = os.path.join(KEY, 'tonal-works-420505-eda807c7cc52.json')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_GOOGLE_APPLICATION_CREDENTIALS

# DATA

In [273]:
# raw
raw_sense_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
raw_sense_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,NaN,This,this,DET,NaN
1,d001.s001.t001,document,document,NOUN,bn:00028015n
2,NaN,is,be,VERB,NaN
3,NaN,a,a,DET,NaN
4,d001.s001.t002,summary,summary,NOUN,bn:00075142n


In [274]:
sentence_df = pd.read_csv(RAW_SENTENCE_TSV, delimiter='\t',  quoting=csv.QUOTE_NONE)
sentence_df.head()
sents = sentence_df["ID"].tolist()
len(sents)

121

In [275]:
sids = sentence_df["ID"].tolist()
# sids = [i.replace('d', 'i') for i in sids]
# sids[:4]
print(sids)

['d001.s001', 'd001.s002', 'd001.s003', 'd001.s004', 'd001.s005', 'd001.s006', 'd001.s007', 'd001.s008', 'd001.s009', 'd001.s010', 'd001.s011', 'd001.s012', 'd001.s013', 'd001.s014', 'd001.s015', 'd001.s016', 'd001.s017', 'd001.s018', 'd001.s019', 'd001.s020', 'd001.s021', 'd001.s022', 'd001.s023', 'd001.s024', 'd001.s025', 'd001.s026', 'd001.s027', 'd001.s028', 'd001.s029', 'd001.s030', 'd001.s031', 'd001.s032', 'd001.s033', 'd001.s034', 'd001.s035', 'd001.s036', 'd001.s037', 'd001.s038', 'd001.s039', 'd001.s040', 'd001.s041', 'd001.s042', 'd001.s043', 'd001.s044', 'd001.s045', 'd001.s046', 'd001.s047', 'd001.s048', 'd001.s049', 'd001.s050', 'd001.s051', 'd001.s052', 'd001.s053', 'd001.s054', 'd001.s055', 'd001.s056', 'd001.s057', 'd001.s058', 'd001.s059', 'd001.s060', 'd001.s061', 'd001.s062', 'd001.s063', 'd001.s064', 'd001.s065', 'd001.s066', 'd001.s067', 'd001.s068', 'd001.s069', 'd001.s070', 'd001.s071', 'd001.s072', 'd001.s073', 'd001.s074', 'd001.s075', 'd001.s076', 'd001.s077'

In [276]:
trans_df = pd.read_csv(TRANS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
trans_df.head()

,ID,Sentence
0,d001.s001,En el presente documento se resume el Informe ...
1,d001.s002,En él se explica cómo el Comité de Medicamento...
2,d001.s003,Si desea más información sobre su enfermedad o...
3,d001.s004,Si desea más información sobre el fundamento e...
4,d001.s005,¿Qué es Alimta ?


In [277]:
tkns_df = pd.read_csv(TOKENS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
tkns_df.head(1000)

,Token ID,Token,Lemma,POS
0,d001.s001.t001,En,NaN,X
1,d001.s001.t002,el,NaN,X
2,d001.s001.t003,presente,presente,J
3,d001.s001.t004,documento,documento,N
4,d001.s001.t005,se,NaN,X
...,...,...,...,...
995,d001.s039.t007,usar,usar,V
996,d001.s039.t008,kalgebra,kalgebra,N
997,d001.s039.t009,.,NaN,X
998,d001.s039.t010,kalgebra,kalgebra,N


In [278]:
tgt_tkns = tkns_df['Token'].tolist()
len(tgt_tkns)

2852

# ALIGNMENT

In [279]:
device = 'cpu'
layer = 8
# "bert": "bert-base-multilingual-cased",
# "xlmr": "xlm-roberta-base"
model = 'xlmr'
# mwmf (Match), inter (ArgMax), itermax (IterMax)
mode = 'inter'
matching_methods = 'mai'
token_type='bpe'
aligner = SentenceAligner(
    model=model
    , token_type=token_type
    , matching_methods=matching_methods
    , layer=layer
    , device=device
)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-07-29 16:51:18,880 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base


In [280]:
raw_sense_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,NaN,This,this,DET,NaN
1,d001.s001.t001,document,document,NOUN,bn:00028015n
2,NaN,is,be,VERB,NaN
3,NaN,a,a,DET,NaN
4,d001.s001.t002,summary,summary,NOUN,bn:00075142n


In [281]:
# english token sequences
idxes = raw_sense_df['Token ID'].tolist()
tokens = raw_sense_df['Token'].tolist()
# print(idxes,tokens)
en_sents = []
# en_sents2 = trans_df["English"].tolist()
# pre_sid = None
sent = []
for i, s in zip(idxes, tokens):
    if s != "-end-":
        sent.append(s)
    else:
        en_sents.append(sent)
        sent = []
print(en_sents)
# for x in range(len(en_sents)):
#     print(" ".join(en_sents[x]),en_sents2[x])
#     if " ".join(en_sents[x]) != en_sents2[x]:
#         print(x)

[['This', 'document', 'is', 'a', 'summary', 'of', 'the', 'European', 'Public', 'Assessment', 'Report', '(', 'EPAR', ')', '.'], ['It', 'explains', 'how', 'the', 'Committee', 'for', 'Medicinal', 'Products', 'for', 'Human', 'Use', '(', 'CHMP', ')', 'assessed', 'the', 'studies', 'performed', ',', 'to', 'reach', 'their', 'recommendations', 'on', 'how', 'to', 'use', 'the', 'medicine', '.'], ['If', 'you', 'need', 'more', 'information', 'about', 'your', 'medical', 'condition', 'or', 'your', 'treatment', ',', 'read', 'the', 'Package', 'Leaflet', '(', 'also', 'part', 'of', 'the', 'EPAR', ')', 'or', 'contact', 'your', 'doctor', 'or', 'pharmacist', '.'], ['If', 'you', 'want', 'more', 'information', 'on', 'the', 'basis', 'of', 'the', 'CHMP', 'recommendations', ',', 'read', 'the', 'Scientific', 'Discussion', '(', 'also', 'part', 'of', 'the', 'EPAR', ')', '.'], ['What', 'is', 'Alimta', '?'], ['Alimta', 'is', 'a', 'powder', 'that', 'is', 'made up', 'into', 'a', 'solution', 'for', 'infusion', '(', 'dri

In [282]:
print(en_sents[92])

['The', 'perspectives', 'of', 'the', 'social', 'partners', 'is', 'also', 'being', 'examined', ',', 'with', 'particular', 'attention', 'paid', 'to', 'proposals', 'for', 'removing', 'age', 'barriers', '.']


In [283]:
# Italian token sequences
idxes = tkns_df['Token ID'].tolist()
tokens = tkns_df['Token'].tolist()

tgt_sents = []
pre_sid = None
s = []
for i, t in zip(idxes, tokens):
    sid = i.split('.')[1]
    if sid != pre_sid:
        if s:
            tgt_sents.append(s)
        pre_sid = sid
        s = [t]
    else:
        s.append(t)
tgt_sents.append(s)
len(tgt_sents)
print(tgt_sents)

[['En', 'el', 'presente', 'documento', 'se', 'resume', 'el', 'Informe', 'Público', 'Europeo', 'de', 'Evaluación', '(', 'EPAR', ')', '.'], ['En', 'él', 'se', 'explica', 'cómo', 'el', 'Comité', 'de', 'Medicamentos', 'de', 'Uso', 'Humano', '(', 'CHMP', ')', 'ha', 'evaluado', 'los', 'estudios', 'realizados', 'con', 'el', 'medicamento', 'a fin de', 'emitir', 'unas', 'recomendaciones', 'sobre', 'su', 'uso', '.'], ['Si', 'desea', 'más', 'información', 'sobre', 'su', 'enfermedad', 'o', 'el', 'tratamiento', 'de', 'la', 'misma', ',', 'le', 'aconsejamos', 'que', 'lea', 'el', 'prospecto', '(', 'incluido', 'en', 'el', 'EPAR', ')', 'o', 'pregunte', 'a', 'su', 'médico', 'o', 'su', 'farmacéutico', '.'], ['Si', 'desea', 'más', 'información', 'sobre', 'el', 'fundamento', 'en', 'el', 'que', 'se', 'han', 'basado', 'las', 'recomendaciones', 'del', 'CHMP', ',', 'le', 'aconsejamos', 'que', 'lea', 'el', 'Debate', 'Científico', '(', 'también', 'incluido', 'en', 'el', 'EPAR', ')', '.'], ['¿Qué', 'es', 'Alimta',

In [284]:
print(tgt_sents[92])

['Está', 'examinando', 'asimismo', 'los', 'puntos', 'de', 'vista', 'de', 'los', 'interlocutores', 'sociales', ',', 'prestando', 'especial', 'atención', 'a', 'las', 'propuestas', 'para', 'eliminar', 'dichos', 'obstáculos', '.']


In [285]:
# do the alignment
aligns = []
for en_s, tgt_s in zip(tqdm(en_sents), tgt_sents):
    # print(en_s,tgt_s,'\n')
    alignments = aligner.get_word_aligns(en_s, tgt_s)
    align = ''
    for w1, w2 in alignments[mode]:
        align += f'{w1}-{w2} '
    align = align.strip()
    aligns.append(align)

100%|██████████| 121/121 [00:58<00:00,  2.07it/s]


In [286]:
aligns[0]

'0-2 1-3 2-4 6-6 7-9 8-8 9-11 10-7 11-12 12-13 13-14 14-15'

In [287]:
print(len(aligns))
trans_df['English-'+language+' Alignment'] = aligns
trans_df.head()

121


,ID,Sentence,English-Spanish Alignment
0,d001.s001,En el presente documento se resume el Informe ...,0-2 1-3 2-4 6-6 7-9 8-8 9-11 10-7 11-12 12-13 ...
1,d001.s002,En él se explica cómo el Comité de Medicamento...,0-1 1-3 2-4 3-5 4-6 5-7 6-8 6-11 7-8 8-9 9-11 ...
2,d001.s003,Si desea más información sobre su enfermedad o...,0-0 2-1 3-2 4-3 5-4 6-5 8-6 9-7 11-9 12-13 13-...
3,d001.s004,Si desea más información sobre el fundamento e...,0-0 2-1 3-2 4-3 5-4 6-5 7-6 10-16 11-14 12-17 ...
4,d001.s005,¿Qué es Alimta ?,0-0 1-1 2-2 3-3


In [288]:
# output
print(len(aligns))
trans_df['English-'+language+' Alignment'] = aligns
trans_df.head()

121


,ID,Sentence,English-Spanish Alignment
0,d001.s001,En el presente documento se resume el Informe ...,0-2 1-3 2-4 6-6 7-9 8-8 9-11 10-7 11-12 12-13 ...
1,d001.s002,En él se explica cómo el Comité de Medicamento...,0-1 1-3 2-4 3-5 4-6 5-7 6-8 6-11 7-8 8-9 9-11 ...
2,d001.s003,Si desea más información sobre su enfermedad o...,0-0 2-1 3-2 4-3 5-4 6-5 8-6 9-7 11-9 12-13 13-...
3,d001.s004,Si desea más información sobre el fundamento e...,0-0 2-1 3-2 4-3 5-4 6-5 7-6 10-16 11-14 12-17 ...
4,d001.s005,¿Qué es Alimta ?,0-0 1-1 2-2 3-3


In [289]:
trans_df.to_csv(ALIGN_SENT_FILE, sep='\t', index=False)

# ALIGN TOKENS IN TOKEN FILE

In [290]:
tkns_df.head()


,Token ID,Token,Lemma,POS
0,d001.s001.t001,En,NaN,X
1,d001.s001.t002,el,NaN,X
2,d001.s001.t003,presente,presente,J
3,d001.s001.t004,documento,documento,N
4,d001.s001.t005,se,NaN,X


In [291]:
aligned_tks_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
aligned_tks_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,NaN,This,this,DET,NaN
1,d001.s001.t001,document,document,NOUN,bn:00028015n
2,NaN,is,be,VERB,NaN
3,NaN,a,a,DET,NaN
4,d001.s001.t002,summary,summary,NOUN,bn:00075142n


In [292]:
aligned_tks_df['Token ID '+tgt_lan] = None
aligned_tks_df['Token '+tgt_lan] = None
aligned_tks_df['Lemma '+tgt_lan] = None
aligned_tks_df['POS '+tgt_lan] = None
aligned_tks_df['Sense '+tgt_lan] = None

In [293]:
aligned_tks_df.head()
print(len(aligned_tks_df['Token']))

2725


In [294]:
trans_df.head()

,ID,Sentence,English-Spanish Alignment
0,d001.s001,En el presente documento se resume el Informe ...,0-2 1-3 2-4 6-6 7-9 8-8 9-11 10-7 11-12 12-13 ...
1,d001.s002,En él se explica cómo el Comité de Medicamento...,0-1 1-3 2-4 3-5 4-6 5-7 6-8 6-11 7-8 8-9 9-11 ...
2,d001.s003,Si desea más información sobre su enfermedad o...,0-0 2-1 3-2 4-3 5-4 6-5 8-6 9-7 11-9 12-13 13-...
3,d001.s004,Si desea más información sobre el fundamento e...,0-0 2-1 3-2 4-3 5-4 6-5 7-6 10-16 11-14 12-17 ...
4,d001.s005,¿Qué es Alimta ?,0-0 1-1 2-2 3-3


In [297]:
def process_nan(s):
  if type(s)!=type(''):
    return ''
  return s

In [298]:
count_en = 0
sentence_num = 0
count_es = 0
for i, row in trans_df.iterrows():
    # print(row.iloc[0])
    sent_i = row.iloc[0]
    # print(sent_i)
    # print(sent_i)
    tgt_sent_i = sent_i
    # print(row)
    # sent_i = sent_i.replace("i","d")
    tgt_sent = tkns_df.loc[count_es:count_es+len(tgt_sents[sentence_num])-1]
    en_sent = aligned_tks_df.loc[count_en:count_en+len(en_sents[sentence_num])-1]
    # print(tgt_sent.loc[2])
    for alignment in row['English-'+language+' Alignment'].split():
        w1, w2 = alignment.split('-')
        row_tgt = tgt_sent.loc[count_es+int(w2)]
        row_en = en_sent.loc[count_en+int(w1)]
        if row_en['Token ID '+tgt_lan] is not None:
            aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] += '➕' + str(row_tgt['Token ID'])
            aligned_tks_df.loc[count_en+int(w1), 'Token '+tgt_lan] += '➕' + row_tgt['Token']
            if aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] != process_nan(row_tgt['POS']):
                aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] += '➕' + process_nan(row_tgt['POS'])
            if aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] != process_nan(row_en['BN Synset']):
                if str(aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan]) != "nan":
                    aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] += '➕' + process_nan(row_en['BN Synset'])
        else:
            aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] = row_tgt['Token ID']
            aligned_tks_df.loc[count_en+int(w1), 'Token '+tgt_lan] = row_tgt['Token']
            aligned_tks_df.loc[count_en+int(w1), 'Lemma '+tgt_lan] = row_tgt["Lemma"]
            aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] = row_tgt['POS']
            aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] = row_en['BN Synset']

    count_en += len(en_sents[sentence_num])+1
    count_es += len(tgt_sents[sentence_num])
    sentence_num += 1
    # print(count_en)

In [299]:
# aligned_tks_df[]
aligned_tks_df.head(30)

,Token ID,Token,Lemma,POS,BN Synset,Token ID ES,Token ES,Lemma ES,POS ES,Sense ES
0,NaN,This,this,DET,NaN,d001.s001.t003,presente,presente,J,NaN
1,d001.s001.t001,document,document,NOUN,bn:00028015n,d001.s001.t004,documento,documento,N,bn:00028015n
2,NaN,is,be,VERB,NaN,d001.s001.t005,se,NaN,X,NaN
3,NaN,a,a,DET,NaN,None,None,None,None,None
4,d001.s001.t002,summary,summary,NOUN,bn:00075142n,None,None,None,None,None
5,NaN,of,of,ADP,NaN,None,None,None,None,None
6,NaN,the,the,DET,NaN,d001.s001.t007,el,NaN,X,NaN
7,d001.s001.t003,European,european,ADJ,bn:00102440a,d001.s001.t010,Europeo,europeo,J,bn:00102440a
8,d001.s001.t004,Public,public,ADJ,bn:00109211a,d001.s001.t009,Público,público,J,bn:00109211a
9,d001.s001.t005,Assessment,assessment,NOUN,bn:00006502n,d001.s001.t012,Evaluación,evaluación,N,bn:00006502n


In [300]:
aligned_tks_df.to_csv(ALIGN_TOKEN_FILE, sep='\t', index=False)